## Imports

In [1]:
# DO NOT CHANGE THESE LINES.
suppressWarnings(
  suppressMessages({
    library(tidyverse)
    library(jsonlite)
    library(fastDummies)
    library(nnet)
  })
)

## Paths

In [2]:
# DO NOT CHANGE THESE LINES.
ROOT_DIR <- dirname(getwd())
MODEL_INPUTS_OUTPUTS <- file.path(ROOT_DIR, 'model_inputs_outputs')
INPUT_DIR <- file.path(MODEL_INPUTS_OUTPUTS, "inputs")
OUTPUT_DIR <- file.path(MODEL_INPUTS_OUTPUTS, "outputs")
INPUT_SCHEMA_DIR <- file.path(INPUT_DIR, "schema")
DATA_DIR <- file.path(INPUT_DIR, "data")
TRAIN_DIR <- file.path(DATA_DIR, "training")
TEST_DIR <- file.path(DATA_DIR, "testing")
MODEL_PATH <- file.path(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH <- file.path(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE <- file.path(MODEL_ARTIFACTS_PATH, 'ohe.rds')
PREDICTOR_DIR_PATH <- file.path(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH <- file.path(PREDICTOR_DIR_PATH, "predictor.rds")
IMPUTATION_FILE <- file.path(MODEL_ARTIFACTS_PATH, 'imputation.rds')
PREDICTIONS_DIR <- file.path(OUTPUT_DIR, 'predictions')
PREDICTIONS_FILE <- file.path(PREDICTIONS_DIR, 'predictions.csv')
LABEL_ENCODER_FILE <- file.path(MODEL_ARTIFACTS_PATH, 'label_encoder.rds')
ENCODED_TARGET_FILE <- file.path(MODEL_ARTIFACTS_PATH, "encoded_target.rds")
TOP_3_CATEGORIES_MAP <- file.path(MODEL_ARTIFACTS_PATH, "top_3_map.rds")

if (!dir.exists(PREDICTIONS_DIR)) {
  dir.create(PREDICTIONS_DIR, recursive = TRUE)
}

### Reading the schema

In [3]:
file_name <- list.files(INPUT_SCHEMA_DIR, pattern = "*.json")[1]
schema <- fromJSON(file.path(INPUT_SCHEMA_DIR, file_name))
features <- schema$features
numeric_features <- features$name[features$dataType != 'CATEGORICAL']
categorical_features <- features$name[features$dataType == 'CATEGORICAL']
id_feature <- schema$id$name
target_feature <- schema$target$name
model_category <- schema$modelCategory
nullable_features <- features$name[features$nullable == TRUE]


### Reading test data.

In [4]:
# Reading test data.
file_name <- list.files(TEST_DIR, pattern = "*.csv", full.names = TRUE)[1]
# Read the first line to get column names
header_line <- readLines(file_name, n = 1)
col_names <- unlist(strsplit(header_line, split = ",")) # assuming ',' is the delimiter
# Read the CSV with the exact column names
df <- read.csv(file_name, skip = 0, col.names = col_names, check.names=FALSE)

## Data preprocessing
Note that when we work with testing data, we have to impute using the same values learned during training. This is to avoid data leakage.

In [5]:
imputation_values <- readRDS(IMPUTATION_FILE)
for (column in names(df)[sapply(df, function(col) any(is.na(col)))]) {
  df[, column][is.na(df[, column])] <- imputation_values[[column]]
}

In [6]:
# Saving the id column in a different variable and then dropping it.
ids <- df[[id_feature]]
df[[id_feature]] <- NULL
head(df)

,act_combined_midrange,act_english_midrange,act_math_midrange,act_writing_midrange,admission_rate,agege24,average_cost_academic_year,average_cost_program_year,carnegie_basic_classification,carnegie_size,⋯,religious_affiliation,sat_math_midrange,sat_total_average,sat_verbal_midrange,sat_writing_midrange,spend_per_student,state,tuition_(instate),tuition_(out_of_state),undergrad_size
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,⋯,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,23,22,22,7,0.6976,0.71,22933,21205.0,Associate\s--Private For-profit',Very small 2-year (less than 500),⋯,,520,1035.5,514,510,6669,CA,11736,14498,277
2,23,22,22,7,0.6976,0.00,17300,17828.5,"Special Focus Institutions--Theological seminaries, Bible colleges, and other faith-related institutions",,⋯,Jewish,520,1035.5,514,510,8516,FL,8400,8400,51
3,23,22,22,7,0.6976,0.47,15178,17828.5,Associate\s--Public Urban-serving Multicampus',"Medium 2-year (2000 to 4,999)",⋯,,520,1035.5,514,510,5065,CA,1142,5942,5707
4,23,22,22,7,0.6976,0.46,22933,24908.0,,,⋯,,520,1035.5,514,510,7178,TX,11736,14498,29
5,23,22,22,7,0.6976,0.64,22933,21500.0,,,⋯,,520,1035.5,514,510,5155,NY,11736,14498,37
6,22,22,21,7,0.7313,0.14,18466,17828.5,Master\s Colleges and Universities (medium programs)',"Medium 4-year, primarily residential (3,000 to 9,999)",⋯,,520,1030.0,514,510,6506,MI,9037,14113,7959


### Encoding
We encode the data using the same encoder that we saved during training.

In [7]:
if (length(categorical_features) > 0 && file.exists(OHE_ENCODER_FILE)) {
  top_3_map <- readRDS(TOP_3_CATEGORIES_MAP)
  encoder <- readRDS(OHE_ENCODER_FILE)
  for(col in categorical_features) {
    # Use the saved top 3 categories to replace values outside these categories with 'Other'
    df[[col]][!(df[[col]] %in% top_3_map[[col]])] <- "Other"
  }

  test_df_encoded <- dummy_cols(df, select_columns = categorical_features, remove_selected_columns = TRUE)
  encoded_columns <- readRDS(OHE_ENCODER_FILE)
  # Add missing columns with 0s
    for (col in encoded_columns) {
        if (!col %in% colnames(test_df_encoded)) {
            test_df_encoded[[col]] <- 0
        }
    }

# Remove extra columns
    extra_cols <- setdiff(colnames(test_df_encoded), c(colnames(df), encoded_columns))
    df <- test_df_encoded[, !names(test_df_encoded) %in% extra_cols]
}

### Making predictions
Using the model saved during training.

In [8]:
# Making predictions
model <- readRDS(PREDICTOR_FILE_PATH)
predictions <- predict(model, newdata = df)

Warning message in predict.lm(model, newdata = df):
“prediction from rank-deficient fit; attr(*, "non-estim") has doubtful cases”


### Creating predictions the Dataframe.

In [9]:
predictions_df <- data.frame(prediction = predictions)
predictions_df <- tibble(ids = ids) %>% bind_cols(predictions_df)
colnames(predictions_df)[1] <- id_feature

write.csv(predictions_df, PREDICTIONS_FILE, row.names = FALSE)